In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "hyper-drive-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Following based on Lesson 'The Azure ML SDK' Exercise 8
# Verify that the cluster does not exist already
try:
    compute_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_D2_V2', 
        max_nodes=4
    )

    compute_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
# Arguments specified in train.py
# C (float): Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.
# max_iter (int): Maximum number of iterations taken for the solvers to converge.
ps = RandomParameterSampling( {
    "--C" : uniform(0.1,1),
    "--max_iter" : choice(50, 75, 100, 125, 150)
    }
)

# Specify a Policy
policy = BanditPolicy(
    evaluation_interval=2, 
    slack_factor=0.1,
    delay_evaluation=4
)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
# This is the estimator for the train script
src = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    compute_target=compute_cluster,
    environment=sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparamter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4
)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(
    config=hyperdrive_config,
    show_output=True
)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()

best_hyperdrive_run_metrics = best_hyperdrive_run.get_metrics()
hyperdrive_parameter_values = best_hyperdrive_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_hyperdrive_run.id)
print('\n Accuracy:', best_hyperdrive_run_metrics['accuracy'])
print('\n Hyperparameters:', hyperdrive_parameter_values)

# Use joblib for saving the model
joblib.dump(best_hyperdrive_run, 'outputs/best_hyperdrive_model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.sklearn.model_selection import train_test_split

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(
        path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv',
)

# Using a TabularDatasetFactory method to split data into training and test
training_data, test_data = dataset.random_split(
        percentage=0.8
)

# alternative see: https://learn.microsoft.com/en-us/azure/machine-learning/v1/how-to-auto-train-models-v1
# x_train, x_test = train_test_split(x, y, test_size=0.2, random_state=2)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(training_data)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=x,
    label_column_name="y",
    n_cross_validations=4
)

In [2]:
# Submit your automl run

automl_exp = Experiment(ws, 'automl_experiment')
automl_run = automl_exp.submit(
    config=automl_config,
    show_output=True
)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.

import joblib
# Get your best run and save the model from that run.
best_automl_run = automl_run.get_best_run_by_primary_metric()

best_automl_run_metrics = best_automl_run.get_metrics()
automl_parameter_values = best_automl_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_automl_run.id)
print('\n Accuracy:', best_automl_run_metrics['accuracy'])
print('\n Hyperparameters:', automl_parameter_values)

# Use joblib for saving the model
joblib.dump(best_automl_run, 'outputs/best_automl_model.joblib')